In [1]:
import sys
import pandas as pd
import numpy as np
import sklearn

In [2]:
#Read csv data using pandas
#data=pd.read_csv('dataset.csv')

In [3]:
#Create a dataframe to contain our processsed data with custom columns
processed_data = pd.DataFrame(columns=['url','evil','length','periods','semicolons','etc'])
#To print
processed_data

,url,evil,length,periods,semicolons,etc
